In [ ]:
! pip3 install pandas
! pip3 install yfinance
! pip3 install plotly
! pip3 install AutoTS

## Previsão de preços de criptomoedas usando Python

Iniciarei a tarefa de previsão de preços de criptomoedas importando as bibliotecas Python necessárias e o conjunto de dados de que precisamos.
Para esta tarefa, coletarei os dados mais recentes de preços do Bitcoin do Yahoo Finance, usando a API yfinance.


In [ ]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

No código acima, coletei os dados mais recentes dos preços do Bitcoin nos últimos 730 dias e os preparei para qualquer analise.
Agora, vamos dar uma olhada na forma desse conjunto de dados para ver se estamos trabalhando com 730 linhas ou não

In [4]:
data.shape

(731, 7)

Portanto, o conjunto de dados contém 731 linhas, onde a primeira linha contém os nomes de cada coluna.
Agora vamos visualizar a mudança nos preços do bitcoin até hoje usando um gráfico de velas

In [ ]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

A coluna Fechar no conjunto de dados contém os valores que precisamos prever,Então, vamos dar uma olhada na correlação de todas as colunas nos dados referentes à coluna Fecha

In [ ]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

## Modelo de previsão de preço de criptomoeda

A previsão dos preços futuros da criptomoeda é baseada no problema da análise de séries temporais
Usaremos a lib autoTS do python, eu uma das melhores libs para este tipo de predição.
vamos utiliza-la para uma previsão de 30 dias.

In [7]:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



Inferred frequency is: D
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Model Number: 6 with model DatepartRegression in generation 0 of 10


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



...

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning:

invalid value encountered in log

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:1223: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:1227: RuntimeWarning:

invalid value encountered in log

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:1223: RuntimeWarning:

divide by zero encountered in true_divide



123 - GLM with avg smape 9.73: 
Model Number: 124 of 129 with model GLM for Validation 3
124 - GLM with avg smape 9.73: 
Model Number: 125 of 129 with model DatepartRegression for Validation 3
125 - DatepartRegression with avg smape 26.57: 
Model Number: 126 of 129 with model GLS for Validation 3
126 - GLS with avg smape 15.77: 
Model Number: 127 of 129 with model UnobservedComponents for Validation 3
127 - UnobservedComponents with avg smape 23.68: 
Model Number: 128 of 129 with model GLM for Validation 3
128 - GLM with avg smape 7.14: 
Model Number: 129 of 129 with model GLM for Validation 3
129 - GLM with avg smape 9.12: 


/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:1223: RuntimeWarning:

divide by zero encountered in true_divide

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done 200 out of 200 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)

                   Close
2022-06-15  20586.310967
2022-06-16  19985.635497
2022-06-17  19846.889482
2022-06-18  19955.831275
2022-06-19  19495.252018
2022-06-20  19315.874542
2022-06-21  19055.789324
2022-06-22  18535.482232
2022-06-23  18213.348223
2022-06-24  18183.623321
2022-06-25  18024.166123
2022-06-26  17916.546041
2022-06-27  17619.331370
2022-06-28  17143.356500
2022-06-29  16846.939033
2022-06-30  16643.652315
2022-07-01  16496.521161
2022-07-02  16374.595729
2022-07-03  16257.572619
2022-07-04  16195.614303
2022-07-05  16214.404450
2022-07-06  16143.922921
2022-07-07  16172.714082
2022-07-08  16066.197490
2022-07-09  16031.350888
2022-07-10  16118.934765
2022-07-11  16361.829692
2022-07-12  16376.586797
2022-07-13  16488.281344
2022-07-14  16602.782523


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    0.0s finished
